Introduction ...

Les imports nécessaires

In [1]:
from time import time
import tensorflow as tf
import numpy as np

from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Importer le dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0
# train_images, test_images = np.mean(train_images, axis=3), np.mean(test_images, axis=3)
# train_images = np.reshape(train_images, (train_images.shape[0], 32, 32))
# test_images  = np.reshape(test_images, (test_images.shape[0], 32, 32))

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

## Classification supervisée sur CIFAR-10

Bla bla ...

### Avec un réseau dense

Bla bla

In [ ]:
model_dense = models.Sequential()
model_dense.add(layers.Flatten(input_shape=(32, 32, 3)))
model_dense.add(layers.Dense(1024, activation='relu'))
model_dense.add(layers.Dense(512, activation='relu'))
model_dense.add(layers.Dense(256, activation='relu'))
model_dense.add(layers.Dense(128, activation='relu'))
model_dense.add(layers.Dense(10))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model_dense.summary()
model_dense.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model_dense.fit(train_images, train_labels, epochs=2,
                    validation_data=(test_images, test_labels))

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model_dense.evaluate(test_images, test_labels, verbose=2)
print(test_acc)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              3146752   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 3,837,066
Trainable params: 3,837,066
Non-trainable params: 0
______________________________________________

### Avec des CNNs

Bla bla

In [ ]:
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='relu'))
model_cnn.add(layers.Dense(10))

In [ ]:
model_cnn.summary()
model_cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model_cnn.fit(train_images, train_labels, epochs=5,
                    validation_data=(test_images, test_labels))

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model_cnn.evaluate(test_images, test_labels, verbose=2)
print(test_acc)

## Auto-encodeurs et applications diverses

Je vais sûrement virer le réseau dense et garder que le CNN ici, vu que les CNNs auront été introduit ci-dessus.

### Avec un réseau dense

Bla bla ...

In [ ]:
latent_dim = 512

class AutoencoderDense(Model):
    def __init__(self, latent_dim):
        super().__init__()
        
        self.latent_dim = latent_dim
        
        self.encoder = tf.keras.Sequential([
            layers.Flatten(),
            layers.Dense(1024, activation='relu'),
            layers.Dense(latent_dim, activation='relu')
        ])

        self.decoder = tf.keras.Sequential([
            layers.Dense(1024, activation='relu'),
            layers.Dense(32 * 32 * 3, activation='sigmoid'),
            layers.Reshape((32, 32, 3))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


### Avec des CNNs

Bla bla ...

In [ ]:
class AutoencoderCNN(Model):
    def __init__(self, latent_dim):
        super().__init__()
        
        self.latent_dim = latent_dim
        
        self.encoder = tf.keras.Sequential([
            layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
            layers.MaxPooling2D((2, 2), padding='same'),
            layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2), padding='same'),
            # layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
            # layers.MaxPooling2D((2, 2), padding='same'),
            layers.Flatten(),
        ])

        self.decoder = tf.keras.Sequential([
            layers.Reshape((8, 8, 16)),
            # layers.UpSampling2D((2, 2)),
            # layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
            layers.UpSampling2D((2, 2)),
            layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
            layers.UpSampling2D((2, 2)),
            layers.Conv2D(3, (3, 3), padding='same')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


Entrainement

In [ ]:
train = True 
autoencoder_cnn = None
if train:
    autoencoder_cnn = AutoencoderCNN(latent_dim)
    autoencoder_cnn.compile(optimizer='adam', loss=losses.MeanSquaredError())

    autoencoder_cnn.fit(train_images, train_images,
                    epochs=1,
                    shuffle=True,
                    validation_data=(test_images, test_images))

    autoencoder_cnn.save('./models/model_cnn1')
else:
    autoencoder_cnn = models.load_model('./models/model_cnn1')
    results = autoencoder_cnn.evaluate(test_images, test_images, verbose=2)
    print('test loss : ', results)

Voir ce que l'auto-encodeur comprend.

In [ ]:
encoded_imgs = autoencoder_cnn.encoder(test_images).numpy()
decoded_imgs = autoencoder_cnn.decoder(encoded_imgs).numpy()

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(test_images[i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Analyse des caractéristiques profondes de l'auto-encodeur et clustering

Le clustering qui n'en est pas encore ...

In [ ]:
train_data = autoencoder_cnn.encoder(train_images)
test_data = autoencoder_cnn.encoder(test_images)

train_labels = train_labels.reshape((50000,))
test_labels = test_labels.reshape((10000,))

### Analyse des caractéristiques

#### Visualisation (t-SNE)

(à résumer)


t-SNE is a great piece of Machine Learning but one can find many reasons to use PCA instead of it. Of the top of my head, I will mention five. As most other computational methodologies in use, t

-SNE is no silver bullet and there are quite a few reasons that make it a suboptimal choice in some cases. Let me mention some points in brief:

    Stochasticity of final solution. PCA is deterministic; t

-SNE is not. One gets a nice visualisation and then her colleague gets another visualisation and then they get artistic which looks better and if a difference of 0.03% in the KL(P||Q) divergence is meaningful... In PCA the correct answer to the question posed is guaranteed. t

-SNE might have multiple minima that might lead to different solutions. This necessitates multiple runs as well as raises questions about the reproducibility of the results.

Interpretability of mapping. This relates to the above point but let's assume that a team has agreed in a particular random seed/run. Now the question becomes what this shows... t

-SNE tries to map only local / neighbours correctly so our insights from that embedding should be very cautious; global trends are not accurately represented (and that can be potentially a great thing for visualisation). On the other hand, PCA is just a diagonal rotation of our initial covariance matrix and the eigenvectors represent a new axial system in the space spanned by our original data. We can directly explain what a particular PCA does.

Application to new/unseen data. t
-SNE is not learning a function from the original space to the new (lower) dimensional one and that's a problem. On that matter, t-SNE is a non-parametric learning algorithm so approximating with parametric algorithm is an ill-posed problem. The embedding is learned by directly moving the data across the low dimensional space. That means one does not get an eigenvector or a similar construct to use in new data. In contrast, using PCA the eigenvectors offer a new axes system what can be directly used to project new data. [Apparently one could try training a deep-network to learn the t

-SNE mapping (you can hear Dr. van der Maaten at ~46' of this video suggesting something along this lines) but clearly no easy solution exists.]

Incomplete data. Natively t
-SNE does not deal with incomplete data. In fairness, PCA does not deal with them either but numerous extensions of PCA for incomplete data (eg. probabilistic PCA) are out there and are almost standard modelling routines. t-SNE currently cannot handle incomplete data (aside obviously training a probabilistic PCA first and passing the PC scores to t

-SNE as inputs).

The k
is not (too) small case. t-SNE solves a problem known as the crowding problem, effectively that somewhat similar points in higher dimension collapsing on top of each other in lower dimensions (more here). Now as you increase the dimensions used the crowding problem gets less severe ie. the problem you are trying to solve through the use of t-SNE gets attenuated. You can work around this issue but it is not trivial. Therefore if you need a k dimensional vector as the reduced set and k is not quite small the optimality of the produce solution is in question. PCA on the other hand offer always the k best linear combination in terms of variance explained. (Thanks to @amoeba for noticing I made a mess when first trying to outline this point.)

#### Réduction de dimension (PCA)

In [ ]:
from sklearn.decomposition import PCA
# PCA
t0 = time()
pca = PCA(n_components=.9, svd_solver='full', whiten=True).fit(train_data)
print(f"Done in {time() - t0}s")

print(pca.components_.shape)

train_data_pca = pca.transform(train_data)
test_data_pca = pca.transform(test_data)

#### Quelques autres algos

### Classification supervisée sur l'espace latent

#### K-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# KNN 
clf = KNeighborsClassifier(5)

clf.fit(train_data_pca, train_labels)
predicted = clf.predict(test_data_pca)


#### Métriques (précision, confusion, etc)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

acc = accuracy_score(test_labels, predicted)

print(acc)
conf_mat = confusion_matrix(test_labels, predicted)


print(conf_mat)

### Classification non supervisée (clustering)

#### K-mean

#### Clustering agglomératif

#### Score NMI